# Handling Email (sending / receiving)

In [1]:
import smtplib

Most of the time, you'll be connecting to either your ISP's SMTP server, or some other server with authentication. In order to avoid exposing my password and such, I'll just use a local proxy server to print some debugging information:

(You can run the proxy with `python -m smtpd -n -c DebuggingServer`)

In [2]:
addr_from = 'rick@arborian.com'
addr_to = 'rick446@usa.net'
msg = '''Subject: This is a test
From: <{}>
To: <{}>

Test message
'''.format(addr_from, addr_to)

print(msg)

Subject: This is a test
From: <rick@arborian.com>
To: <rick446@usa.net>

Test message



In [3]:
conn = smtplib.SMTP('localhost', 8025)
conn.set_debuglevel(1)
# conn.login(username, password)
conn.sendmail(addr_from, [addr_to], msg)

send: 'ehlo theodin.localdomain\r\n'
reply: b'250-theodin.localdomain\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250 HELP\r\n'
reply: retcode (250); Msg: b'theodin.localdomain\n8BITMIME\nHELP'
send: 'mail FROM:<rick@arborian.com>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Subject: This is a test\r\nFrom: <rick@arborian.com>\r\nTo: <rick446@usa.net>\r\n\r\nTest message\r\n.\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
data: (250, b'OK')


{}

## Creating and parsing email messages

If you're going to be putting together more complicated emails, the `email` module is extremely useful (for multipart messages, attachments, etc.):

In [4]:
from email.mime.text import MIMEText

In [5]:
msg = MIMEText('This is a text message')
msg['Subject'] = 'This is a test message'
msg['From'] = addr_from
msg['To'] = addr_to
print(msg.as_string())

Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit
Subject: This is a test message
From: rick@arborian.com
To: rick446@usa.net

This is a text message


In [6]:
conn.sendmail(addr_from, [addr_to], msg.as_string())

send: 'mail FROM:<rick@arborian.com>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Content-Type: text/plain; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\nSubject: This is a test message\r\nFrom: rick@arborian.com\r\nTo: rick446@usa.net\r\n\r\nThis is a text message\r\n.\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
data: (250, b'OK')


{}

In [8]:
from email.mime.multipart import MIMEMultipart

In [9]:
msg = MIMEMultipart()
msg['Subject'] = 'This is a test message'
msg['From'] = addr_from
msg['To'] = addr_to
text_part = MIMEText('This is a text message', 'plain')
html_part = MIMEText('<h1>This is an HTML message</h1>', 'html')
msg.attach(text_part)
msg.attach(html_part)
print(msg.as_string())

Content-Type: multipart/mixed; boundary="===============5420140365372035895=="
MIME-Version: 1.0
Subject: This is a test message
From: rick@arborian.com
To: rick446@usa.net

--===============5420140365372035895==
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

This is a text message
--===============5420140365372035895==
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

<h1>This is an HTML message</h1>
--===============5420140365372035895==--



In [11]:
conn.sendmail(addr_from, [addr_to], msg.as_string())

send: 'mail FROM:<rick@arborian.com>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Content-Type: multipart/mixed; boundary="===============5420140365372035895=="\r\nMIME-Version: 1.0\r\nSubject: This is a test message\r\nFrom: rick@arborian.com\r\nTo: rick446@usa.net\r\n\r\n--===============5420140365372035895==\r\nContent-Type: text/plain; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\n\r\nThis is a text message\r\n--===============5420140365372035895==\r\nContent-Type: text/html; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\n\r\n<h1>This is an HTML message</h1>\r\n--===============5420140365372035895==--\r\n.\r\n'
reply: b'250 OK\r\n'
re

{}

In [12]:
from email.parser import Parser

In [13]:
parsed = Parser().parsestr(msg.as_string())

In [14]:
parsed

In [15]:
headers = dict(parsed)
headers

{'Content-Type': 'multipart/mixed; boundary="===============5420140365372035895=="',
 'MIME-Version': '1.0',
 'Subject': 'This is a test message',
 'From': 'rick@arborian.com',
 'To': 'rick446@usa.net'}

In [16]:
for part in parsed.walk():
    if part.is_multipart():
        continue
    print('>>> Begin part (class = {}) <<<'.format(part.__class__))
    print(dict(part))
    print(part)
    print('>>> End part <<<')
    print('')

>>> Begin part (class = <class 'email.message.Message'>) <<<
{'Content-Type': 'text/plain; charset="us-ascii"', 'MIME-Version': '1.0', 'Content-Transfer-Encoding': '7bit'}
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

This is a text message
>>> End part <<<

>>> Begin part (class = <class 'email.message.Message'>) <<<
{'Content-Type': 'text/html; charset="us-ascii"', 'MIME-Version': '1.0', 'Content-Transfer-Encoding': '7bit'}
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

<h1>This is an HTML message</h1>
>>> End part <<<



In [17]:
part.get_payload()

'<h1>This is an HTML message</h1>'

## Accessing email on the server using `imaplib`

In [20]:
import imaplib
server = imaplib.IMAP4_SSL('imap.gmail.com', 993)
server.login('class@arborian.com', 'class-test1')

('OK', [b'class@arborian.com authenticated (Success)'])

In [21]:
server.select()
typ, data = server.search(None, 'ALL')
for num in data[0].split():
    typ, data = server.fetch(num, '(RFC822)')
    msg = Parser().parsestr(str(data[0][1], 'utf-8'))
    print('Message {}\n{}\n'.format(num, dict(msg)))

Message b'1'
{'MIME-Version': '1.0', 'x-no-auto-attachment': '1', 'Received': 'by 2002:a05:651c:235:0:0:0:0; Thu, 21 May 2020 15:14:57 -0700 (PDT)', 'Date': 'Thu, 21 May 2020 15:14:57 -0700', 'Message-ID': '<CAArXQHG3_OtZwKvjuBysFvxxx5Xmf4=ZSdur_wnq4OhoN42YDA@mail.gmail.com>', 'Subject': 'Get the official Gmail app', 'From': 'Gmail Team <mail-noreply@google.com>', 'To': 'Class Account <class@arborian.com>', 'Content-Type': 'multipart/alternative; boundary="000000000000cb37c305a62fd7ea"'}

Message b'2'
{'MIME-Version': '1.0', 'x-no-auto-attachment': '1', 'Received': 'by 2002:a05:651c:235:0:0:0:0; Thu, 21 May 2020 15:14:57 -0700 (PDT)', 'Date': 'Thu, 21 May 2020 15:14:57 -0700', 'Message-ID': '<CAArXQHGAOt+Mq-Q=8Est6TcjkLg0WXO1uVJB34+qNNRBVYY1wA@mail.gmail.com>', 'Subject': 'Tips for using your new inbox', 'From': 'Gmail Team <mail-noreply@google.com>', 'To': 'Class Account <class@arborian.com>', 'Content-Type': 'multipart/alternative; boundary="000000000000cff9ba05a62fd760"'}

Message b

Message b'6'
{'Delivered-To': 'class@arborian.com', 'Received': 'by 2002:a05:6638:1b0:0:0:0:0 with SMTP id b16csp472832jaq;\r\n        Wed, 16 Dec 2020 10:15:08 -0800 (PST)', 'X-Received': 'by 2002:a67:ef03:: with SMTP id j3mr32738626vsr.26.1608142507920;\r\n        Wed, 16 Dec 2020 10:15:07 -0800 (PST)', 'ARC-Seal': 'i=1; a=rsa-sha256; t=1608142507; cv=none;\r\n        d=google.com; s=arc-20160816;\r\n        b=ZqkAaZD+OWY84sSmD7oHPDISt1aa39NblthoFtpJsB72+tqEibPuZmY0KqhvXTNAEJ\r\n         Yrkep7xv6QhPqTo24Fjt3TUSz8i85p9Wd3E3o5LadEW+NnzGQNS2Znh8T4zhCAR6HCqT\r\n         NA5snF10FT/onE7lNg1U8myPC6UTSFXahPNwEtrS4cEghUI1hWa2wku7PJvECEwmv5dX\r\n         pOk1pT6OLqf24nvRrCJamVyoIqiM/+t3Bf6bSjG4YSBZZGAbh9XfivHo8gnkCpUQe0ld\r\n         XAYMB/KFQ0uSQPwtWJqa+4Bz9i/Mr1NgxaAIT1Hgm7NsiRQhccx+608egF0xKh8iPxfo\r\n         Mmbw==', 'ARC-Message-Signature': 'i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;\r\n        h=to:from:subject:message-id:feedback-id:date:mime-version\r\n     

In [22]:
server.close()
server.logout()

('BYE', [b'LOGOUT Requested'])

# Lab

Open [Email Lab](./email-lab.ipynb)